In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torchsummary import summary
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.utils import clip_grad_norm_

import numpy as np
import gzip
import pickle
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gc
import time
import random
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from collections import deque
import pandas as pd
import math


import sys
sys.path.append('..')
# from slp_package.slp_functions import create_merged_game_data_df
from slp_package.input_dataset import InputDataSet
import slp_package.pytorch_functions as slp_pytorch_functions

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using CUDA
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
torch.cuda.is_available()

True

In [2]:
source_data = ['ranked','public','mango']

general_features = {
    'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM','DREAMLAND'],
    'num_players': [2],
    'conclusive': [True],
}
player_features = {
    # 'netplay_code': ['MANG#0'],
    # 'character_name': ['FALCO'],
    'character_name': ['PIKACHU'],
    # 'character_name': ['FOX', 'FALCO', 'MARTH', 'CAPTAIN_FALCON', 'SHEIK'],
    # 'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'MARIO', 'PEACH', 'ROY', 'LUIGI', 'YOUNG_LINK', 'DONKEY_KONG', 'PICHU', 'KIRBY'],
    # 'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'PEACH', 'LUIGI', 'DONKEY_KONG'],
    'type_name': ['HUMAN']
    
}
opposing_player_features = {
    # 'character_name': ['MARTH'],
    # 'netplay_code': ['KOD#0', 'ZAIN#0']
    'type_name': ['HUMAN']
}
label_info = {
    'source': ['player'], # Can be 'general', 'player
    # 'feature': ['netplay_code']
    'feature': ['character_name']
}

In [3]:
dataset = InputDataSet(source_data, general_features, player_features, opposing_player_features, label_info)

print(dataset.dataset['labels'].value_counts())
dataset.dataset.head()

PIKACHU    4096
Name: labels, dtype: int64


/workspace/slp_jaspar/anomaly_detection_minute/../slp_package/input_dataset.py:95: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  processed_df = pd.concat([player_1_df, player_2_df], ignore_index=True)


,stage_name,num_players,conclusive,player_character_name,player_type_name,opposing_player_type_name,player_inputs_np_sub_path,length,labels
0,FINAL_DESTINATION,2,True,PIKACHU,HUMAN,HUMAN,mango\PIKACHU\daa0e300-d731-42ea-a634-28c9db71...,7244,PIKACHU
1,POKEMON_STADIUM,2,True,PIKACHU,HUMAN,HUMAN,mango\PIKACHU\142d032c-4d4a-4056-b9a9-aca37c88...,7384,PIKACHU
2,YOSHIS_STORY,2,True,PIKACHU,HUMAN,HUMAN,mango\PIKACHU\3a508695-3da7-464a-b228-88d0f051...,14390,PIKACHU
3,FOUNTAIN_OF_DREAMS,2,True,PIKACHU,HUMAN,HUMAN,mango\PIKACHU\1be3107a-58a6-4f48-80c0-191cf926...,11363,PIKACHU
4,BATTLEFIELD,2,True,PIKACHU,HUMAN,HUMAN,mango\PIKACHU\0b446825-d7e3-405d-9b67-33a80ba1...,9797,PIKACHU


In [4]:
labels_order =  dataset.number_of_segments_per_game(3600,15000)
print(labels_order)
labels_order = labels_order['Label'].values

     Label  Count  Shift
0  PIKACHU   4067   2032


In [5]:
train_df, pikachu_test_df  = dataset.train_test_split_dataframes(test_ratio = .20, val = False)

                           player_inputs_np_sub_path  length  num_segments  \
0  public\PIKACHU\683b5d15-bc36-4c91-8503-aff1dee...    9693             3   
1  ranked\PIKACHU\5d30f16a-a1cb-44c7-b815-e03724e...   11724             4   
2  public\PIKACHU\c337441e-e286-4703-ae91-16c32d0...   11724             4   
3  ranked\PIKACHU\d881f727-966a-44e1-a9c6-d4d37ee...    7650             2   
4  ranked\PIKACHU\5b9fa78a-09a3-4c75-9901-3da6bed...   13746             5   

    labels  encoded_labels  
0  PIKACHU               0  
1  PIKACHU               0  
2  PIKACHU               0  
3  PIKACHU               0  
4  PIKACHU               0  


In [6]:
train_df, pikachu_test_df = dataset.all_segments_train_test_split_dataframes(3600, proportion_of_segments=1, test_ratio = .2, val = False)
# porportion = .8
# train_df = train_df.sample(frac=porportion, random_state = 42)
# porportion = .05
# pikachu_test_df = pikachu_test_df.sample(frac=porportion, random_state = 42)

In [7]:
print(train_df.shape)
print(pikachu_test_df.shape)

train_df.head()

(8408, 6)
(2098, 6)


,player_inputs_np_sub_path,labels,encoded_labels,segment_start_index,segment_index,segment_length
0,ranked\PIKACHU\68fcf66a-c7ba-4a5d-9f96-bbe9cf8...,PIKACHU,0,0,0,3600
1,ranked\PIKACHU\68fcf66a-c7ba-4a5d-9f96-bbe9cf8...,PIKACHU,0,2978,1,3600
2,ranked\PIKACHU\68fcf66a-c7ba-4a5d-9f96-bbe9cf8...,PIKACHU,0,5956,2,3600
3,ranked\PIKACHU\68fcf66a-c7ba-4a5d-9f96-bbe9cf8...,PIKACHU,0,8934,3,3600
4,ranked\PIKACHU\f3d5e964-c520-45c5-b589-2eac95d...,PIKACHU,0,0,0,3600


In [8]:
source_data = ['ranked','public','mango']

general_features = {
    'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM','DREAMLAND'],
    'num_players': [2],
    'conclusive': [True],
}
player_features = {
    # 'netplay_code': ['MANG#0'],
    # 'character_name': ['FALCO'],
    'character_name': ['PICHU'],
    # 'character_name': ['FOX', 'FALCO', 'MARTH', 'CAPTAIN_FALCON', 'SHEIK'],
    # 'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'MARIO', 'PEACH', 'ROY', 'LUIGI', 'YOUNG_LINK', 'DONKEY_KONG', 'PICHU', 'KIRBY'],
    # 'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'PEACH', 'LUIGI', 'DONKEY_KONG'],
    'type_name': ['HUMAN']
    
}
opposing_player_features = {
    # 'character_name': ['MARTH'],
    # 'netplay_code': ['KOD#0', 'ZAIN#0']
    'type_name': ['HUMAN']
}
label_info = {
    'source': ['player'], # Can be 'general', 'player
    # 'feature': ['netplay_code']
    'feature': ['character_name']
}

dataset = InputDataSet(source_data, general_features, player_features, opposing_player_features, label_info)

print(dataset.dataset['labels'].value_counts())
dataset.dataset.head()



PICHU    230
Name: labels, dtype: int64


/workspace/slp_jaspar/anomaly_detection_minute/../slp_package/input_dataset.py:95: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  processed_df = pd.concat([player_1_df, player_2_df], ignore_index=True)


,stage_name,num_players,conclusive,player_character_name,player_type_name,opposing_player_type_name,player_inputs_np_sub_path,length,labels
0,BATTLEFIELD,2,True,PICHU,HUMAN,HUMAN,mango\PICHU\36e5d441-a59c-411f-a381-c233d8b5e3...,7817,PICHU
1,YOSHIS_STORY,2,True,PICHU,HUMAN,HUMAN,mango\PICHU\33ab85fd-cdf4-4fc1-8e09-9830156d11...,8924,PICHU
2,BATTLEFIELD,2,True,PICHU,HUMAN,HUMAN,mango\PICHU\6e1d9b21-c812-48a9-bbbe-f86432c909...,9863,PICHU
3,FOUNTAIN_OF_DREAMS,2,True,PICHU,HUMAN,HUMAN,ranked\PICHU\0651785c-667e-4c03-ad6f-16315e25f...,1052,PICHU
4,BATTLEFIELD,2,True,PICHU,HUMAN,HUMAN,ranked\PICHU\0532f85a-bc4e-4c19-8c5f-aa767408e...,12718,PICHU


In [9]:
labels_order =  dataset.number_of_segments_per_game(3600,500)
print(labels_order)
labels_order = labels_order['Label'].values

   Label  Count  Shift
0  PICHU    227   3296


In [10]:
train_df, pichu_test_df = dataset.all_segments_train_test_split_dataframes(3600, proportion_of_segments=1, test_ratio = .99, val = False)
# porportion = .8
# train_df = train_df.sample(frac=porportion, random_state = 42)
# porportion = .05
# pichu_test_df = pichu_test_df.sample(frac=porportion, random_state = 42)
print(pichu_test_df.shape)


(563, 6)


In [11]:
test_df = pd.concat([pichu_test_df,pikachu_test_df], ignore_index=True)
test_df['labels'].value_counts()

PIKACHU    2098
PICHU       563
Name: labels, dtype: int64

In [12]:
class TrainingDataset(Dataset):
    """
    Custom dataset for loading game segments from compressed numpy files.
    """
    def __init__(self, df, transform=None):
        self.file_paths = df['player_inputs_np_sub_path'].to_numpy()
        self.encoded_labels = df['encoded_labels'].to_numpy()
        self.segment_start_index = df['segment_start_index'].to_numpy()
        # self.segment_index = df['segment_index'].to_numpy()
        self.segment_length = df['segment_length'].to_numpy()
        self.transform = transform

    def __len__(self):
        """Returns the total number of samples in the dataset."""
        return len(self.file_paths)

    def __getitem__(self, idx):
        """Loads and returns a sample from the dataset at the specified index."""
        with gzip.open('/workspace/melee_project_data/input_np/' + self.file_paths[idx].replace('\\','/'), 'rb') as f:
            segment = np.load(f)
        
        # Start and end of the segment
        segment_start = self.segment_start_index[idx]
        segment_end = self.segment_start_index[idx] + self.segment_length[idx]
        
        segment = segment[:,int(segment_start):int(segment_end)]
        
        # separate into positive and negative values
        # if self.transform:
        #     transformed = np.zeros((13,60))
        #     transformed[0,:] = (np.abs(segment[0]) - .2875) * 1.40350877193 * (segment[0] > 0)
        #     transformed[1,:] = (np.abs(segment[0]) - .2875) * 1.40350877193 * (segment[0] < 0)
        #     transformed[2,:] = (np.abs(segment[1]) - .2875) * 1.40350877193 * (segment[1] > 0)
        #     transformed[3,:] = (np.abs(segment[1]) - .2875) * 1.40350877193 * (segment[1] < 0)
        #     transformed[4,:] = (np.abs(segment[2]) - .2875) * 1.40350877193 * (segment[2] > 0)
        #     transformed[5,:] = (np.abs(segment[2]) - .2875) * 1.40350877193 * (segment[2] < 0)
        #     transformed[6,:] = (np.abs(segment[3]) - .2875) * 1.40350877193 * (segment[3] > 0)
        #     transformed[7,:] = (np.abs(segment[3]) - .2875) * 1.40350877193 * (segment[3] < 0)
        #     transformed[8:,:] = segment[4:]
        #     segment = transformed
        
        # if self.transform:
        #     # Shift inputs: adjusting so that positives are reduced and negatives are increased
        #     segment[0:4, :] -= .2875 * (segment[0:4, :] > 0)
        #     segment[0:4, :] += .2875 * (segment[0:4, :] < 0)

        #     # Scale inputs to be between -.5 and .5
        #     segment[0:4, :] *= 1.40350877193 / 2
        #     segment[0:4, :] += .5
        segment[-5] = (segment[-5] > .5)
        
        if self.transform:
            transformed = np.zeros((9 + 5+4,60))
            
            # First 4 rows of transformed are the analog inputs transformed to have range [0,1]
            # Shift inputs: adjusting so that positives are reduced and negatives are increased
            analog_transformed = np.copy(segment[0:4])
            analog_transformed[analog_transformed > 0] -= 0.2875 + 0.0125
            analog_transformed[analog_transformed < 0] += 0.2875 - 0.0125
            # Scale inputs to be between -.5 and .5
            analog_transformed *= .5 / .725
            # Add .5 to so final inputs are between 0 and 1
            analog_transformed += .5
            
            transformed[0:4] = analog_transformed
            # Next four rows are 1 if the corresponding analog input is 0
            transformed[4:8] += (segment[:4] == 0)
            
            
            prepend = np.expand_dims(segment[-5:, 0], axis=1)
            transitions= np.abs(np.diff(segment[-5:], axis=1, prepend=prepend))
            
            transformed[8:13] += transitions
            
            

            # Transform the Trigger so that it is 0 or 1
            
            
            # Remaining rows are button inputs
            transformed[-5:] += segment[-5:]
            
            
        
        # Convert to PyTorch tensors
        segment_tensor = torch.from_numpy(transformed).float()
        # label_tensor = torch.tensor(self.encoded_labels[idx], dtype=torch.long)
        return segment_tensor#, label_tensor
    
class TrainingDataset(Dataset):
    """
    Custom dataset for loading game segments from compressed numpy files.
    """
    def __init__(self, df, transform=None):
        self.file_paths = df['player_inputs_np_sub_path'].to_numpy()
        self.encoded_labels = df['encoded_labels'].to_numpy()
        self.segment_start_index = df['segment_start_index'].to_numpy()
        # self.segment_index = df['segment_index'].to_numpy()
        self.segment_length = df['segment_length'].to_numpy()
        self.transform = transform

    def __len__(self):
        """Returns the total number of samples in the dataset."""
        return len(self.file_paths)

    def __getitem__(self, idx):
        """Loads and returns a sample from the dataset at the specified index."""
        with gzip.open('/workspace/melee_project_data/input_np/' + self.file_paths[idx].replace('\\','/'), 'rb') as f:
            segment = np.load(f)
        
        # Start and end of the segment
        segment_start = self.segment_start_index[idx]
        segment_end = self.segment_start_index[idx] + self.segment_length[idx]
        
        segment = segment[:,int(segment_start):int(segment_end)]
        
        # separate into positive and negative values
        # if self.transform:
        #     transformed = np.zeros((13,60))
        #     transformed[0,:] = (np.abs(segment[0]) - .2875) * 1.40350877193 * (segment[0] > 0)
        #     transformed[1,:] = (np.abs(segment[0]) - .2875) * 1.40350877193 * (segment[0] < 0)
        #     transformed[2,:] = (np.abs(segment[1]) - .2875) * 1.40350877193 * (segment[1] > 0)
        #     transformed[3,:] = (np.abs(segment[1]) - .2875) * 1.40350877193 * (segment[1] < 0)
        #     transformed[4,:] = (np.abs(segment[2]) - .2875) * 1.40350877193 * (segment[2] > 0)
        #     transformed[5,:] = (np.abs(segment[2]) - .2875) * 1.40350877193 * (segment[2] < 0)
        #     transformed[6,:] = (np.abs(segment[3]) - .2875) * 1.40350877193 * (segment[3] > 0)
        #     transformed[7,:] = (np.abs(segment[3]) - .2875) * 1.40350877193 * (segment[3] < 0)
        #     transformed[8:,:] = segment[4:]
        #     segment = transformed
        
        # if self.transform:
        #     # Shift inputs: adjusting so that positives are reduced and negatives are increased
        #     segment[0:4, :] -= .2875 * (segment[0:4, :] > 0)
        #     segment[0:4, :] += .2875 * (segment[0:4, :] < 0)

        #     # Scale inputs to be between -.5 and .5
        #     segment[0:4, :] *= 1.40350877193 / 2
        #     segment[0:4, :] += .5
        segment[-5] = (segment[-5] > .5)
        
        if self.transform:
            transformed = np.zeros((9 + 5+4,3600))
            
            # First 4 rows of transformed are the analog inputs transformed to have range [0,1]
            # Shift inputs: adjusting so that positives are reduced and negatives are increased
            analog_transformed = np.copy(segment[0:4])
            analog_transformed[analog_transformed > 0] -= 0.2875 + 0.0125
            analog_transformed[analog_transformed < 0] += 0.2875 - 0.0125
            # Scale inputs to be between -.5 and .5
            analog_transformed *= .5 / .725
            # Add .5 to so final inputs are between 0 and 1
            analog_transformed += .5
            
            transformed[0:4] = analog_transformed
            # Next four rows are 1 if the corresponding analog input is 0
            transformed[4:8] += (segment[:4] == 0)
            
            
            prepend = np.expand_dims(segment[-5:, 0], axis=1)
            transitions= np.abs(np.diff(segment[-5:], axis=1, prepend=prepend))
            
            transformed[8:13] += transitions
            
            

            # Transform the Trigger so that it is 0 or 1
            
            
            # Remaining rows are button inputs
            transformed[-5:] += segment[-5:]
            
            
        
        # Convert to PyTorch tensors
        segment_tensor = torch.from_numpy(transformed).float()
        # label_tensor = torch.tensor(self.encoded_labels[idx], dtype=torch.long)
        return segment_tensor#, label_tensor
    
def prepare_data_loaders(train_df, test_df, batch_size, num_workers):
    # Initialize datasets
    train_dataset = TrainingDataset(train_df,True)
    # val_dataset = TrainingDataset(file_paths_val, labels_val)
    test_dataset = TrainingDataset(test_df,True)

    # Initialize data loaders
    loaders = {
        'train': DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True, pin_memory=True,persistent_workers=True),
        'test': DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, pin_memory=True,persistent_workers=True),
        # 'val': DataLoader(val_dataset, batch_size=2**9, num_workers=num_workers, shuffle=False, pin_memory=True,persistent_workers=True)
    }
    return loaders



# ''' Get a batch of data to see the size if we want that information. ''' 
# data_loader_iterator = iter(loaders['train'])
# first_batch = next(data_loader_iterator)
# print(first_batch.shape)



In [13]:

def train_model(model, criterion, optimizer, loaders, device, num_epochs=1):
    # Initialize progress bar for the epochs
    epoch_progress = tqdm(range(num_epochs), desc='Training Progress', unit='epoch')

    for epoch in epoch_progress:
        model.train()
        epoch_loss_sum = 0
        epoch_total = 0

        for _, target_cpu in enumerate(loaders['train']):
            # Move data to the appropriate device
            target_gpu = target_cpu.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            output_gpu = model(target_gpu)
            
            # Calculate loss
            loss = criterion(output_gpu, target_gpu) / (18 * 3600 * target_cpu.size(0))
            loss.backward()
            
            # Clip gradients to avoid exploding gradients
            # clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            # Perform a single optimization step
            optimizer.step()
            
            # Accumulate loss sum for accurate average calculation
            epoch_loss_sum += loss.item() * target_cpu.size(0)
            epoch_total += target_cpu.size(0)

        # Calculate average loss for the epoch
        epoch_loss = epoch_loss_sum / epoch_total

        # Update progress bar with the final loss of the epoch
        epoch_progress.set_postfix(Loss=f'{epoch_loss * 100:.10f}')
        print(epoch_loss* 100)

    return model

def evaluate_model(model, criterion, loader, device):
    model.eval()
    eval_loss = False
    total = 0
    
    with torch.no_grad():
        eval_loader_tqdm = tqdm(loader, unit = 'batch')
        
        for batch_number, target_cpu in enumerate(eval_loader_tqdm):
            target_gpu = target_cpu.to(device)
            output_gpu = model(target_gpu)
            
            if batch_number == 0:
                eval_loss = criterion(output_gpu, target_gpu)
            else:
                eval_loss = torch.cat((eval_loss, criterion(output_gpu, target_gpu)),0)
            
            
            # total += target_gpu.shape[0] / (32 * 16 * 4)
            # eval_loader_tqdm.set_postfix(loss=f'{eval_loss / (total):.10f}') 
    
    return eval_loss.to('cpu').numpy()
            
    # print(f'Evaluated Loss: {eval_loss / total:.10f}')=
    

    

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torchsummary import summary
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau

import numpy as np
import gzip
import pickle
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gc
import time
import random
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from collections import deque
import pandas as pd
import math


import sys
sys.path.append('..')
# from slp_package.slp_functions import create_merged_game_data_df
from slp_package.input_dataset import InputDataSet
import slp_package.pytorch_functions as slp_pytorch_functions

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using CUDA
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
torch.cuda.is_available()

from Convolutional_Autoencoder_Model import ResNet_Autoencoder


channels = 9 + 5 + 4

# Build model
model = ResNet_Autoencoder(channels)
model.load_state_dict(torch.load('/workspace/melee_project_data/pikachu_convolutional_autoencoder_1_minute_bottlenecksize_15_64_2222_weights_3.pt'))
model.to('cuda')
# # With the size of an input we can get a model summary.
summary(model, input_size=(channels, 3600))

# Check that the output shape and target shape match
# training_example = torch.rand(9, 2 ** 12).to('cuda')
# print('Target shape:', training_example.shape)
# model.eval()
# output = model(training_example)
# print('Output shape:', output.shape)

## Optionally compile the model
# import torch_tensorrt
# model = torch.compile(model, mode = 'default')
# model = torch.compile(model,mode = 'max-autotune')


# model = torch.compile(model, backend="torch_tensorrt")
# model = torch.compile(model, backend="torch_tensorrt",mode = 'max-autotune')


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [-1, 64, 3600]           1,216
       BatchNorm1d-2             [-1, 64, 3600]             128
              ReLU-3             [-1, 64, 3600]               0
            Conv1d-4             [-1, 64, 3600]          12,352
       BatchNorm1d-5             [-1, 64, 3600]             128
              ReLU-6             [-1, 64, 3600]               0
            Conv1d-7            [-1, 256, 3600]          16,640
       BatchNorm1d-8            [-1, 256, 3600]             512
            Conv1d-9            [-1, 256, 3600]           4,864
      BatchNorm1d-10            [-1, 256, 3600]             512
             ReLU-11            [-1, 256, 3600]               0
Encoder_Bottleneck-12            [-1, 256, 3600]               0
           Conv1d-13             [-1, 64, 3600]          16,448
      BatchNorm1d-14             [-1, 

In [15]:
# import torch._dynamo
# torch._dynamo.config.suppress_errors = True

torch.autograd.set_detect_anomaly(False)
# Pepare data loaders
batch_size =  8
num_workers = 16
loaders = prepare_data_loaders(train_df, pikachu_test_df, batch_size, num_workers)


    
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()
        self.BCE = nn.BCEWithLogitsLoss(reduction='none')  # Consider using weighted BCE if needed
        self.MSE = nn.MSELoss(reduction='none')
        self.bin_threshold = 2
        # Self.bin_threshold bins to the left and right should account for the barely close enough threshold
        self.barely_close_enough = (.5 + self.bin_threshold) * 0.00862#
        self.scale_factor = - math.log(.5) / self.barely_close_enough ** 2
        
    def forward(self, pred, target):
        # Calculating losses
        mse_loss = self.MSE(torch.sigmoid(pred[:,0:4,:]), target[:,0:4,:]) 
        bce_loss = self.BCE(pred[:,4:,:], target[:,4:,:])
        
        # print(mse_loss.shape)
        # print(bce_loss.shape)
        return torch.cat((mse_loss, bce_loss / 100),1)
    


criterion = CustomLoss()

pikachu_loss = evaluate_model(model, criterion, loaders['test'], 'cuda')

loaders = prepare_data_loaders(train_df, pichu_test_df, batch_size, num_workers)

pichu_loss = evaluate_model(model, criterion, loaders['test'], 'cuda')



100%|██████████| 71/71 [00:01<00:00, 57.03batch/s]


In [16]:
pikachu_loss_mean = np.mean(pikachu_loss,axis=2)
pikachu_loss_mean = np.mean(pikachu_loss_mean,axis=1)
print(np.mean(pikachu_loss_mean))

pichu_loss_mean = np.mean(pichu_loss,axis=2)
pichu_loss_mean = np.mean(pichu_loss_mean,axis=1)
print(np.mean(pichu_loss_mean))
print(np.mean(pikachu_loss_mean) - np.mean(pichu_loss_mean))

0.009284254
0.009430248
-0.00014599413


In [17]:
threshold = (np.mean(pikachu_loss_mean) + np.mean(pichu_loss_mean)) / 2
correct_pikachu = np.sum(pikachu_loss_mean < threshold)
correct_pichu = np.sum(pichu_loss_mean > threshold)
print('acc pikachu', correct_pikachu / pikachu_loss.shape[0])
print('acc pichu', correct_pichu / pichu_loss.shape[0])

acc pikachu 0.7354623450905624
acc pichu 0.3552397868561279


In [18]:
pikachu_loss_mean = np.mean(pikachu_loss,axis=2)
pikachu_loss_mean = np.mean(pikachu_loss_mean,axis=0)
# print(pikachu_loss_mean)

pichu_loss_mean = np.mean(pichu_loss,axis=2)
pichu_loss_mean = np.mean(pichu_loss_mean,axis=0)
# print(pichu_loss_mean)

print(pikachu_loss_mean - pichu_loss_mean)
for index in range(18):

    threshold = (pikachu_loss_mean[index] + pichu_loss_mean[index]) / 2
    print(index)
    print('acc pikachu', np.sum(pikachu_loss_mean < threshold) / pikachu_loss.shape[0])
    print('acc pichu', np.sum(pichu_loss_mean > threshold) / pichu_loss.shape[0])

[-8.3097741e-03  3.2034554e-03  1.5211781e-04  2.4693492e-03
  6.3433778e-05  2.7790479e-04  5.7577388e-05  5.0649955e-04
  5.7245023e-05  4.8944770e-05 -1.9644154e-05  2.3087166e-05
  1.1915690e-04 -6.0697272e-04  8.7492226e-05 -4.3492764e-04
 -2.2687612e-04 -9.6115516e-05]
0
acc pikachu 0.008579599618684462
acc pichu 0.0017761989342806395
1
acc pikachu 0.0076263107721639654
acc pichu 0.0017761989342806395
2
acc pikachu 0.005243088655862726
acc pichu 0.010657193605683837
3
acc pikachu 0.006196377502383222
acc pichu 0.0053285968028419185
4
acc pikachu 0.00667302192564347
acc pichu 0.003552397868561279
5
acc pikachu 0.006196377502383222
acc pichu 0.0053285968028419185
6
acc pikachu 0.0014299332697807435
acc pichu 0.02486678507992895
7
acc pikachu 0.0038131553860819827
acc pichu 0.017761989342806393
8
acc pikachu 0.0023832221163012394
acc pichu 0.021314387211367674
9
acc pikachu 0.0
acc pichu 0.03019538188277087
10
acc pikachu 0.0023832221163012394
acc pichu 0.02486678507992895
11
acc pi